In [2]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
#library(mediation)
library(caret)
#library(gbm)
#library(party)
#library(Metrics)
#install.packages("wesanderson")
#library(wesanderson)
#library(gtools)
#library(interactions)
#install.packages('ggstance')
#library(ggstance)
#install.packages('elasticnet')
#library(elasticnet)
#install.packages('kernlab')
#library(kernlab)
#install.packages('e1071')
#library(e1071)
#install.packages('MatchIt')
#library(MatchIt)
library(dplyr)
#library(ukbtools)
#library(mediation)
library(lavaan)
#install.packages('MissMech')
#library(semTools)
#library(MissMech)
#install.packages('fsbrain')
#library(fsbrain)
#install.packages('ipw')
#library(ipw)
#install.packages('episensr')
#install.packages('measurements')
#library(measurements)
#library(tidyr)
#library(optimx)
#install.packages('devtools')
#library(devtools)
#library(reshape)
library(childsds)

## Read the data

In [3]:
ABCD_longitudinal=read.table('/dagher/dagher11/filip/PRSOB/data/ABCD_base_fup1_wide.csv',sep=',',dec='.', header=T)
ABCD=read.table('/dagher/dagher11/filip/PRSOB/data/ABCD_base_fup1_long.csv',sep=',',dec='.', header=T)

In [4]:
ABCD_longitudinal$deltaVol=ABCD_longitudinal$H_site_smri_vol_cdk_total.y-ABCD_longitudinal$H_site_smri_vol_cdk_total.x
ABCD_longitudinal$deltaThick=ABCD_longitudinal$H_site_smri_thick_cdk_mean.y-ABCD_longitudinal$H_site_smri_thick_cdk_mean.x
ABCD_longitudinal$deltaSA=ABCD_longitudinal$H_site_smri_area_cdk_total.y-ABCD_longitudinal$H_site_smri_area_cdk_total.x

In [5]:
ABCD=merge(ABCD, dplyr::select(ABCD_longitudinal, subjectkey, deltaBMI_SD, deltaVol, deltaThick, deltaSA), all.x=T)

In [6]:
PCs=read.table('/dagher/dagher11/filip/Downloads/ABCD_pre-impute.eigenvec', header=T)

## Remove duplicates

In [7]:
ABCD=ABCD[!duplicated(ABCD$subjectkey),]

In [8]:
ABCD=subset(ABCD, ABCD$demo_race_a_p___10==1)

## Remove all participants who failed WM QC for all sample consistency

In [9]:
ABCD=subset(ABCD, ABCD$dmri_dti_postqc_qc==1 | is.na(ABCD$dmri_dti_postqc_qc))

## Read definitions

In [10]:
aliases=read.table('/dagher/dagher11/filip/ABCD_data/abcd_smrip101_definitions.csv',quote='"', header=T, sep=',')
aliases2=read.table('/dagher/dagher11/filip/ABCD_data/abcd_smrip201_definitions.csv',quote='"', header=T, sep=',')

In [11]:
ABCD=ABCD[complete.cases(ABCD$H_dmri_dtifa_fiberat_fxrh),]

In [12]:
new_PRS=read.table('/dagher/dagher11/filip/Downloads/BMI_PRS_new.csv', header=T, sep=',')
colnames(new_PRS)=c( 'X', 'FID' ,'IID' ,'PRS', 'Z_UKB')

In [13]:
ABCD=merge(ABCD, PCs, by.x='subjectkey', by.y='IID', all.x=T)
ABCD=merge(ABCD, new_PRS, by.x='subjectkey', by.y='IID', all.x=T)

## Read in ADI data

In [14]:
data_list=list()

files=c('abcd_rhds01.txt')

for (i in 1:length(files)){
    data = readLines(paste("/dagher/dagher9/filip/ABCD/Package_1203460/",files[i],sep=''))
    skip_second = data[-2]
    data = read.csv(textConnection(skip_second), header = TRUE, stringsAsFactors = FALSE, sep='\t')
    drops <- c("dataset_id", "interview_date",'collection_id','src_subject_id')
    data = data[ , !(names(data) %in% drops)]    
    data_list[[i]]=data
}



In [15]:
data_list[[1]]=data_list[[1]][data_list[[1]]$eventname=='baseline_year_1_arm_1',]

In [16]:
ABCD=merge((ABCD), dplyr::select(data_list[[1]], subjectkey, reshist_addr1_coi_z_ed_nat,
                             reshist_addr1_coi_z_se_nat,
                             reshist_addr1_coi_z_he_nat), by='subjectkey', all.x=T)

## Correct PRS for PCs

In [17]:
ABCD$Z_corr=resid(lm(Z ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, 
                     data=ABCD, na.action=na.exclude))

ABCD$Z_corr_UKB=resid(lm(Z_UKB ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, 
                     data=ABCD, na.action=na.exclude))

In [18]:
nrow(ABCD)

[1] 4157

## Descriptive statistics of the sample

In [364]:
describe(ABCD$age_yrs)
describe(ABCD$BMI)
describe(ABCD$BMI_SD)
describe(ABCD$weight_kg)
describe(ABCD$weight_SD)
table(ABCD$sex)

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
X1,1,4157,9.948621,0.612439,9.916667,9.945797,0.7413,8.916667,11,2.083333,0.01504122,-1.218994,0.009498889


,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
X1,1,4157,17.94186,3.299112,17.1069,17.50303,2.447757,10.77081,38.87181,28.101,1.5681,3.662627,0.05116901


,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
X1,1,4157,0.2016688,1.088399,0.1815334,0.2203325,1.098212,-6.916598,2.834634,9.751232,-0.3458139,0.8025366,0.016881


,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
X1,1,4157,35.60207,8.576444,33.56584,34.52509,6.724961,18.59729,86.48496,67.88767,1.43349,2.911855,0.1330201


,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
X1,1,4157,0.2844252,1.006183,0.2317527,0.2686331,1.005664,-3.316608,3.460271,6.77688,0.1151537,-0.1977742,0.01560584



   F    M 
1940 2217 

## BMI-PRS and BMI SD corelation

In [189]:
summary((lm(ABCD$BMI_SD ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
           PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l + reshist_addr1_coi_z_ed_nat +
           reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat +
           interview_age + interview_age + interview_age*sex , data=ABCD)))

summary(lm(ABCD$BMI_SD ~ Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
           PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l + reshist_addr1_coi_z_ed_nat +
           reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat +
           interview_age + interview_age + interview_age*sex, data=ABCD))


Call:
lm(formula = ABCD$BMI_SD ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + 
    PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + 
    PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l + reshist_addr1_coi_z_ed_nat + 
    reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat + 
    interview_age + interview_age + interview_age * sex, data = ABCD)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.0749 -0.6910  0.0025  0.7582  2.7870 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)   
(Intercept)                 1.061251   0.453674   2.339  0.01937 * 
PC1                         1.763511   1.318234   1.338  0.18105   
PC2                        -2.370410   1.284896  -1.845  0.06514 . 
PC3                         0.407108   1.236152   0.329  0.74192   
PC4                         1.790073   1.210468   1.479  0.13927   
PC5                        -0.654799   1.180192  -0.555  0.57905   
PC6                        -2.678522   1.180021  -2.270  0.02327 


Call:
lm(formula = ABCD$BMI_SD ~ Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + 
    PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + 
    PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + site_id_l + reshist_addr1_coi_z_ed_nat + 
    reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat + 
    interview_age + interview_age + interview_age * sex, data = ABCD)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.3193 -0.6534  0.0193  0.7097  2.6653 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 0.950365   0.437588   2.172   0.0299 *  
Z_UKB                       0.286966   0.016822  17.059   <2e-16 ***
PC1                         0.225690   1.274546   0.177   0.8595    
PC2                        -1.677764   1.239866  -1.353   0.1761    
PC3                        -0.428516   1.193197  -0.359   0.7195    
PC4                         1.891376   1.167435   1.620   0.1053    
PC5                        -0.875950   1.138294  -

# Figure 1

In [19]:
std_mean <- function(x) sd(x)/sqrt(length(x))

ABCD_sum = ABCD %>%  group_by(group = cut(Z_UKB, breaks = seq(-4.5, 4.5, 0.5))) %>%
  summarise(Num = n(), mean = (mean(BMI_SD,na.rm=TRUE)), se=std_mean(BMI_SD))



In [34]:
col_graph='deepskyblue4'

#tiff('/dagher/dagher11/filip/PRSOB/Figures/BMI_SDS_plot.tiff', 
#     width=15, height=15, units='cm', res=300)
fig1a=ggplot() +
geom_bar(mapping=aes(x=ABCD_sum$group, y=ABCD_sum$Num), stat='identity', fill=col_graph, color=col_graph) +
geom_point(mapping=aes(x=ABCD_sum$group, y=ABCD_sum$mean*400)) +
geom_errorbar(data=ABCD_sum, mapping=aes(x=group, y=mean*400, ymin=mean*400-se*400, ymax=mean*400+se*400), 
              width=.6,position=position_dodge(.9)) +
geom_smooth(data=ABCD_sum, mapping=aes(x=as.numeric(group), y=mean*400), method='lm', color='black', se=F, 
            size=0.5) +
scale_y_continuous(name='Number of individuals', breaks=c(0, 100, 200, 300, 400, 500, 600, 700, 800), 
                   sec.axis = sec_axis(~./300, name = "BMI SDS", 
                                       breaks=c(-2.5, -2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5))) +
theme_minimal() +
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=12), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
scale_x_discrete(name='BMI-PRS z-score') 

#dev.off()
  #geom_point(aes(y=BMI_SD))
  #scale_y_continuous(name="cyl", sec.axis=sec_axis(~./scaleFactor, name="hp"))

ABCD_plot1b=select(ABCD, BMI_SD, site_id_l)

ABCD_plot1b=ABCD_plot1b %>%
  group_by(site_id_l) %>%
  summarise(
    mean = mean(BMI_SD),
    sd = sd(BMI_SD),
    n = n(),
    se = sd / n
  ) 

  fig1b=ggplot(data=ABCD_plot1b, aes(x = site_id_l, y = mean)) +
  geom_bar(position = position_dodge(), stat='identity', fill=col_graph, color=col_graph) +
  geom_errorbar(
    aes(ymin = mean - se, ymax = mean + se), 
    position = position_dodge2(padding = 0.5)
  ) +
  labs(
    title = ""
  ) + theme_minimal() + ylab('BMI SDS') + xlab('Site') +
    theme(axis.text.x = element_text(size = 12, angle = 0), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
    scale_x_discrete(labels=c("site01" = "1", "site02" = "2","site03" = "3","site04" = "4","site05" = "5",
                              "site06" = "6","site07" = "7","site08" = "8","site09" = "9","site10" = "10",
                              "site11" = "11","site12" = "12","site13" = "13","site14" = "14","site15" = "15",
                              "site16" = "16","site17" = "17","site18" = "18","site19" = "19","site20" = "20",
                              "site21" = "21", 'site22'='22')) 

ABCD_plot1c=select(ABCD, Z_UKB, site_id_l)

ABCD_plot1c=ABCD_plot1c %>%
  group_by(site_id_l) %>%
  summarise(
    mean = mean(Z_UKB),
    sd = sd(Z_UKB),
    n = n(),
    se = sd / n
  )

  fig1c=ggplot(data=ABCD_plot1c, aes(x = site_id_l, y = mean)) +
  geom_bar(position = position_dodge(), stat='identity', fill=col_graph, color=col_graph) +
  geom_errorbar(
    aes(ymin = mean - se, ymax = mean + se), 
    position = position_dodge2(padding = 0.5)
  ) +
  labs(
    title = ""
  ) + theme_minimal() + ylab('BMI-PRS') + xlab('Site') +
    theme(axis.text.x = element_text(size = 12, angle = 0), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
    scale_x_discrete(labels=c("site01" = "1", "site02" = "2","site03" = "3","site04" = "4","site05" = "5",
                              "site06" = "6","site07" = "7","site08" = "8","site09" = "9","site10" = "10",
                              "site11" = "11","site12" = "12","site13" = "13","site14" = "14","site15" = "15",
                              "site16" = "16","site17" = "17","site18" = "18","site19" = "19","site20" = "20",
                              "site21" = "21", 'site22'='22')) 


ABCD_plot1d=select(ABCD, PC1, site_id_l)

ABCD_plot1d=ABCD_plot1d %>%
  group_by(site_id_l) %>%
  summarise(
    mean = mean(PC1),
    sd = sd(PC1),
    n = n(),
    se = sd / n
  )

  fig1d=ggplot(data=ABCD_plot1d, aes(x = site_id_l, y = mean)) +
  geom_bar(position = position_dodge(), stat='identity', fill=col_graph, color=col_graph) +
  geom_errorbar(
    aes(ymin = mean - se, ymax = mean + se), 
    position = position_dodge2(padding = 0.5)
  ) +
  labs(
    title = ""
  ) + theme_minimal() + ylab('Genetic principal component 1') + xlab('Site') +
    theme(axis.text.x = element_text(size = 12, angle = 0), panel.grid.minor = element_blank(),
     panel.grid.major = element_blank()) +
    scale_x_discrete(labels=c("site01" = "1", "site02" = "2","site03" = "3","site04" = "4","site05" = "5",
                              "site06" = "6","site07" = "7","site08" = "8","site09" = "9","site10" = "10",
                              "site11" = "11","site12" = "12","site13" = "13","site14" = "14","site15" = "15",
                              "site16" = "16","site17" = "17","site18" = "18","site19" = "19","site20" = "20",
                              "site21" = "21", 'site22'='22')) 


In [35]:
library('ggpubr')

tiff('/dagher/dagher11/filip/PRSOB/Figures/Figure1.tiff', 
     width=30, height=30, units='cm', res=300)

figure1 <- ggarrange(fig1a, fig1b, fig1c, fig1d,
                    labels = c("a", "b", "c", "d"),
                    ncol = 2, nrow = 2)
figure1

dev.off()


`geom_smooth()` using formula 'y ~ x'



png 
  2

# SEM

In [34]:
cog=read.table('/dagher/dagher11/filip/PRSOB/data/abcd_execfunc.csv', header=T, sep=',', dec='.')
colnames(cog)=c('subjectkey',colnames(cog)[2:ncol(cog)])
ABCD=merge(ABCD, cog, by='subjectkey',all.x=T)

In [35]:
hdr <- read.table('/export01/data/fmorys/ABCD30/abcd_mhy02.txt', sep='\t',header=T, quote='"', stringsAsFactors = FALSE)
inh3 <- read.table('/export01/data/fmorys/ABCD30/abcd_mhy02.txt', sep='\t',header=F, quote='"', skip = 2)
colnames( inh3 ) <- colnames(hdr)
inh3=subset(inh3, select=-c(sex, collection_title, study_cohort_name, dataset_id, interview_date, 
                          collection_id, src_subject_id, interview_age)) #Remove unnecessary columns for quicker merging further on
inh3=inh3[inh3$eventname=='baseline_year_1_arm_1',]

ABCD=merge(ABCD, inh3, by='subjectkey',all.x=T)

ABCD$H_WM_FA=rowMeans(data.frame(ABCD$H_site_dmri_dtifa_fiberat_allfiblh, ABCD$H_site_dmri_dtifa_fiberat_allfibrh))


In [102]:
ABCD$H_site_smri_vol_cdk_total_PC=resid(lm(ABCD$H_site_smri_vol_cdk_total ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$H_site_smri_thick_cdk_mean_PC=resid(lm(ABCD$H_site_smri_thick_cdk_mean ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$H_WM_FA_PC=resid(lm(ABCD$H_WM_FA ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$BMI_SD_site=resid(lm(ABCD$BMI_SD ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$deltaBMI_SD_site=resid(lm(ABCD$deltaBMI_SD ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + BMI_SD_site, na.action=na.exclude, data=ABCD))
ABCD$z.scores_site=resid(lm(ABCD$z.scores ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$Z_corr_site=resid(lm(ABCD$Z_corr ~ ABCD$site_id_l, na.action=na.exclude))
ABCD$Z_corr_UKB_site=resid(lm(ABCD$Z_corr_UKB ~ ABCD$site_id_l, na.action=na.exclude))
ABCD$upps_y_ss_positive_urgency_site=resid(lm(ABCD$upps_y_ss_positive_urgency ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))
ABCD$upps_y_ss_negative_urgency_site=resid(lm(ABCD$upps_y_ss_negative_urgency ~ ABCD$site_id_l + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, na.action=na.exclude, data=ABCD))

In [103]:
Lavaan_SEM=dplyr::select(ABCD, 
                         H_site_smri_vol_cdk_total_PC,
                        H_site_smri_thick_cdk_mean_PC,
                        H_WM_FA_PC,
                        Z_corr_UKB_site,
                        z.scores_site,
                        upps_y_ss_positive_urgency_site,
                        upps_y_ss_negative_urgency_site,
                        BMI_SD_site,
                        deltaBMI_SD_site,
                        demo_comb_income_v2,
                        demo_ed_v2,
                        demo_prnt_marital_v2,
                        demo_prnt_ed_v2,
                        interview_age,
                        sex)

In [104]:
for (i in 1:8) {
    
    Lavaan_SEM[[i]]=resid(lm(Lavaan_SEM[[i]] ~ interview_age + 
                              sex + 
                              factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                              factor(ABCD$demo_comb_income_v2, ordered=F) + 
                              factor(ABCD$demo_ed_v2, ordered=F) +
                              factor(ABCD$demo_prnt_marital_v2, ordered=F), 
                              data=Lavaan_SEM, na.action=na.exclude))
        
    }
        
Lavaan_SEM$z.scores_site[is.na(Lavaan_SEM$z.scores_site)]=mean(Lavaan_SEM$z.scores_site, na.rm=T)   
Lavaan_SEM$upps_y_ss_negative_urgency_site[is.na(Lavaan_SEM$upps_y_ss_negative_urgency_site)]=mean(Lavaan_SEM$upps_y_ss_negative_urgency_site, na.rm=T)        
Lavaan_SEM$upps_y_ss_positive_urgency_site[is.na(Lavaan_SEM$upps_y_ss_positive_urgency_site)]=mean(Lavaan_SEM$upps_y_ss_positive_urgency_site, na.rm=T)        

In [108]:
ABCD_SEM = '

#regressions

H_site_smri_vol_cdk_total_PC ~ Z_corr_UKB_site
H_site_smri_thick_cdk_mean_PC ~ Z_corr_UKB_site 
H_WM_FA_PC ~ Z_corr_UKB_site 

z.scores_site ~  H_site_smri_vol_cdk_total_PC + H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC 
upps_y_ss_positive_urgency_site ~  H_site_smri_vol_cdk_total_PC + H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC  
upps_y_ss_negative_urgency_site ~  H_site_smri_vol_cdk_total_PC + H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC  

deltaBMI_SD_site ~ z.scores_site + 
        upps_y_ss_positive_urgency_site + upps_y_ss_negative_urgency_site  

#covariance
H_site_smri_vol_cdk_total_PC ~~ H_site_smri_thick_cdk_mean_PC + H_WM_FA_PC
H_site_smri_thick_cdk_mean_PC ~~ H_WM_FA_PC
upps_y_ss_positive_urgency_site ~~ upps_y_ss_negative_urgency_site + z.scores_site
upps_y_ss_negative_urgency_site ~~ z.scores_site


'


ABCD_SEM_model <- sem(ABCD_SEM, data=Lavaan_SEM, missing='listwise', se='robust.sem', estimator='ML')
lavaan:::print.lavaan.summary(summary(ABCD_SEM_model, standardized=TRUE, fit.measures=TRUE))
#fitmeasures(WMHOB_fit) #fit indices

lavaan 0.6-12 ended normally after 36 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        28

  Number of observations                          4157

Model Test User Model:
                                                      
  Test statistic                                55.349
  Degrees of freedom                                 7
  P-value (Chi-square)                           0.000

Model Test Baseline Model:

  Test statistic                              2322.913
  Degrees of freedom                                28
  P-value                                        0.000

User Model versus Baseline Model:

  Comparative Fit Index (CFI)                    0.979
  Tucker-Lewis Index (TLI)                       0.916

Loglikelihood and Information Criteria:

  Loglikelihood user model (H0)             -41624.984
  Loglikelihood unrestricted model (H1)     -415

# Calculate follow-up stats - cortical volume + thickness

In [41]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr.csv', sep=",")

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  uniq

,measure,hemi,estimate,p,t,NA
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
23,precentral,lh,-0.041815384600647,0.00000000,-2.70913511610892,H_site_smri_vol_cdk_precnlh
29,superiortemporal,lh,-0.0286325873085021,0.01224000,-1.85098990747833,H_site_smri_vol_cdk_sutmlh
31,frontalpole,lh,-0.0278704176573553,0.01926667,-1.77225325663824,H_site_smri_vol_cdk_frpolelh
42,inferiortemporal,rh,-0.0340992262989611,0.00000000,-2.23826005371772,H_site_smri_vol_cdk_iftmrh
57,precentral,rh,-0.0336507591257409,0.00000000,-2.22364299208494,H_site_smri_vol_cdk_precnrh
61,superiorfrontal,rh,-0.0370071392381847,0.00000000,-2.68868952719717,H_site_smri_vol_cdk_sufrrh


In [200]:
Stats=matrix(nrow=length(c(grep('H_site_smri_thick_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_thick_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                              reshist_addr1_coi_z_ed_nat +
                              reshist_addr1_coi_z_se_nat + 
                              reshist_addr1_coi_z_he_nat,
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr.csv', sep=",")

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  uniq

,measure,hemi,estimate,p,t,NA
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
19,parstriangularis,lh,-0.0268247136436155,0.02234286,-1.75538043025274,H_site_smri_thick_cdk_parstgrislh
27,superiorfrontal,lh,-0.0316854601607416,0.00000000,-2.25693170966451,H_site_smri_thick_cdk_sufrlh
32,temporalpole,lh,-0.0418379681555957,0.00000000,-2.6323795362324,H_site_smri_thick_cdk_tmpolelh
37,caudalmiddlefrontal,rh,-0.0271133208874662,0.00000000,-1.8675032215052,H_site_smri_thick_cdk_cdmdfrrh
42,inferiortemporal,rh,-0.0253891413983069,0.00000000,-1.7278904011106,H_site_smri_thick_cdk_iftmrh
47,medialorbitofrontal,rh,-0.0252999301998751,0.00000000,-1.66790181763338,H_site_smri_thick_cdk_mobfrrh
60,rostralmiddlefrontal,rh,-0.0275180404597772,0.00000000,-2.0150888424709,H_site_smri_thick_cdk_rrmdfrrh


In [98]:
summary(lmp(BMI_SD ~  site_id_l + 
                     reshist_addr1_coi_z_ed_nat +
                     reshist_addr1_coi_z_se_nat + 
                     reshist_addr1_coi_z_he_nat, 
           data=ABCD, maxIter = 10000, center=F ))

[1] "Settings:  unique SS "



Call:
lmp(formula = BMI_SD ~ site_id_l + reshist_addr1_coi_z_ed_nat + 
    reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat, 
    data = ABCD, center = F, maxIter = 10000)

Residuals:
      Min        1Q    Median        3Q       Max 
-6.991241 -0.688067  0.007866  0.753644  2.698912 

Coefficients:
                            Estimate  Iter Pr(Prob)    
site_id_l1                 -0.058469    51   1.0000    
site_id_l2                 -0.320434 10000   <2e-16 ***
site_id_l3                  0.296590 10000   <2e-16 ***
site_id_l4                 -0.040038    51   1.0000    
site_id_l5                 -0.147852  3962   0.0247 *  
site_id_l6                 -0.044134    51   0.9412    
site_id_l7                  0.039913  1133   0.0812 .  
site_id_l8                 -0.005112    51   1.0000    
site_id_l9                  0.065440    51   1.0000    
site_id_l10                -0.043464   395   0.2025    
site_id_l11                 0.065471  2214   0.0434 *  
site_id_l12        

In [94]:
summary(lmp(Z_UKB ~  site_id_l + 
                     reshist_addr1_coi_z_ed_nat +
                     reshist_addr1_coi_z_se_nat + 
                     reshist_addr1_coi_z_he_nat, 
           data=ABCD, maxIter = 10000, center=F ))

[1] "Settings:  unique SS "



Call:
lmp(formula = Z_UKB ~ site_id_l + reshist_addr1_coi_z_ed_nat + 
    reshist_addr1_coi_z_se_nat + reshist_addr1_coi_z_he_nat, 
    data = ABCD, center = F, maxIter = 10000)

Residuals:
      Min        1Q    Median        3Q       Max 
-4.042639 -0.652246  0.004671  0.656069  3.705410 

Coefficients:
                            Estimate  Iter Pr(Prob)    
site_id_l1                 -0.278531  2130   0.0451 *  
site_id_l2                 -0.064072   221   0.3122    
site_id_l3                  0.210474  1842   0.0516 .  
site_id_l4                  0.003444    51   1.0000    
site_id_l5                  0.029711    51   1.0000    
site_id_l6                 -0.097639  4895   0.0200 *  
site_id_l7                 -0.125698    51   1.0000    
site_id_l8                 -0.058177   726   0.1212    
site_id_l9                  0.041572    51   1.0000    
site_id_l10                -0.173168  9178   0.0108 *  
site_id_l11                -0.010387   161   0.3851    
site_id_l12         

# Mesulam analysis

In [148]:
library('effectsize')


Attaching package: ‘effectsize’


The following object is masked from ‘package:psych’:

    phi




In [149]:
Mesulam=read.table('/dagher/dagher11/filip/Downloads/Mesulam_dkt.csv',sep=';', header=F)
colnames(Mesulam)=c('struct','class')
Mesulam=Mesulam[!(Mesulam$struct=='unknown' | Mesulam$struct=='corpuscallosum'),]
Mesulam
#Mesulam=Mesulam[2:68,]

#1 = paralimbic
#2 = heteromodal
#3 = unimodal
#4 - idiotypic

,struct,class
,<chr>,<int>
2,bankssts,2
3,caudalanteriorcingulate,1
4,caudalmiddlefrontal,3
6,cuneus,4
7,entorhinal,1
8,fusiform,3
9,inferiorparietal,2
10,inferiortemporal,3
11,isthmuscingulate,1


In [150]:
volume=cbind(as.data.frame(cbind(ABCD[c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])])))

In [155]:
Mes_stats=matrix(nrow=4, ncol=5)

index=1

for (i in 1:4) { #
  
    model <- (lm(rowSums(volume[Mesulam$class==i]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20  + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD))
    
    modelperm <- (lmp(rowSums(volume[Mesulam$class==i]) ~
                      ABCD$Z_UKB + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20  + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD, maxIter=10000, center=F))
    
    out=summary(model)
    out2=summary(modelperm)
    eta2_PRS=eta_squared(model)[1,2]
    
    
    name=i
    
    Mes_stats[index,][1]=name#colnames(ABCD[i])
    Mes_stats[index,][2]=eta2_PRS
    Mes_stats[index,][3]=out$coefficients[2,1] # estimate
    Mes_stats[index,][4]=out2$coefficients[2,3] # p-value
    Mes_stats[index,][5]=out$coefficients[2,3] # t-value
       
     
    index=index+1
}

Mes_stats_adj=Mes_stats
Mes_stats_adj[,4]=as.numeric(p.adjust(Mes_stats_adj[,4], method = 'fdr'))
Mes_stats_adj=data.frame(Mes_stats_adj)
colnames(Mes_stats_adj)=c('measure','eta2','estimate','p','t')

Mes_stats_adj[Mes_stats_adj[,4]<0.05,]

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "


,measure,eta2,estimate,p,t
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,3,0.001167045,-0.2029528,0,-1.989336


In [156]:
Mes_stats_adj

measure,eta2,estimate,p,t
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.0002112535,-0.1041056,0.06479626,-1.006488
2,0.0008230013,-0.2519457,0.24961948,-1.837061
3,0.0011670451,-0.2029528,0.00000000,-1.989336
4,0.0001269617,-0.1024724,1.00000000,-1.102818


In [49]:
thickness=cbind(as.data.frame(cbind(ABCD[c(grep('H_site_smri_thick_cdk',colnames(ABCD))[1:68])])))

In [50]:
Mes_stats=matrix(nrow=4, ncol=5)

index=1

for (i in 1:4) { #
  
    model <- (lm(rowMeans(thickness[Mesulam$class==i]) ~
                      ABCD$Z_corr_UKB + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD))
    
    modelperm <- (lmp(rowMeans(thickness[Mesulam$class==i]) ~
                      ABCD$Z_corr_UKB + site_id_l +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$mri_info_manufacturer,ordered=F), 
                                data=ABCD, maxIter=10000, center=F))
    
    out=summary(model)
    out2=summary(modelperm)
    eta2_PRS=eta_squared(model)[1,2]
    
    
    name=i
    
    Mes_stats[index,][1]=name#colnames(ABCD[i])
    Mes_stats[index,][2]=eta2_PRS
    Mes_stats[index,][3]=out$coefficients[2,1] # estimate
    Mes_stats[index,][4]=out2$coefficients[2,3] # p-value
    Mes_stats[index,][5]=out$coefficients[2,3] # t-value
       
     
    index=index+1
}

Mes_stats_adj=Mes_stats
Mes_stats_adj[,4]=as.numeric(p.adjust(Mes_stats_adj[,4], method = 'fdr'))
Mes_stats_adj=data.frame(Mes_stats_adj)
colnames(Mes_stats_adj)=c('measure','eta2','estimate','p','t')

Mes_stats_adj

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "


measure,eta2,estimate,p,t
1,0.0002099542,-0.008739231,0.5737052,-1.1254738
2,0.0002927890,-0.012226265,0.7922705,-1.3404565
3,0.0001499493,-0.008325338,0.5737052,-0.8421846
4,0.0002171581,0.009469719,1.0000000,0.9590336


# Other associations

## BMI SDS - volume

In [32]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$BMI_SD + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$BMI_SD + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr_BMISD.csv', sep=",")

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  uniq

,measure,hemi,estimate,p,t,NA
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
36,caudalanteriorcingulate,rh,0.0400375054843331,0,2.53239532336611,H_site_smri_vol_cdk_cdacaterh


# Positive urgency - volume

In [111]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_positive_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_positive_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr_posurg.csv', sep=",")

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  uniq

,measure,hemi,estimate,p,t,NA
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
3,caudalmiddlefrontal,lh,-0.021342098739264,0.000000000,-3.91596905617377,H_site_smri_vol_cdk_cdmdfrlh
8,inferiortemporal,lh,-0.00940650701602929,0.005666667,-1.82894880133814,H_site_smri_vol_cdk_iftmlh
25,rostralanteriorcingulate,lh,-0.0172553363196199,0.000000000,-3.06259694368761,H_site_smri_vol_cdk_rracatelh
27,superiorfrontal,lh,-0.0145944969069519,0.000000000,-2.91054833594432,H_site_smri_vol_cdk_sufrlh
48,middletemporal,rh,-0.00949079196398087,0.020400000,-1.75170190260179,H_site_smri_vol_cdk_mdtmrh
67,transversetemporal,rh,-0.00827010318168227,0.000000000,-1.54539709036565,H_site_smri_vol_cdk_trvtmrh
68,insula,rh,-0.0192363875035213,0.000000000,-3.65905504608922,H_site_smri_vol_cdk_insularh


# Negative urgency - volume

In [112]:
Stats=matrix(nrow=length(c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])),ncol=6) # 
index=1

for (i in c(grep('H_site_smri_vol_cdk',colnames(ABCD))[1:68])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_negative_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$upps_y_ss_negative_urgency + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
    name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
    hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=name#colnames(ABCD[i])
    Stats[index,][2]=hemi
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr_negurg.csv', sep=",")

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  uniq

,measure,hemi,estimate,p,t,NA
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
3,caudalmiddlefrontal,lh,-0.0175699226746942,0.0000,-2.99687683472823,H_site_smri_vol_cdk_cdmdfrlh
4,cuneus,lh,-0.0135354332636722,0.0000,-2.35071025310302,H_site_smri_vol_cdk_cuneuslh
26,rostralmiddlefrontal,lh,-0.00965199181844791,0.0000,-1.64622761578421,H_site_smri_vol_cdk_rrmdfrlh
28,superiorparietal,lh,-0.00848460155183616,0.0017,-1.46674228215413,H_site_smri_vol_cdk_supllh
29,superiortemporal,lh,-0.0151514895224775,0.0136,-2.58475743972819,H_site_smri_vol_cdk_sutmlh
36,caudalanteriorcingulate,rh,-0.0131117451847518,0.0000,-2.18723091523222,H_site_smri_vol_cdk_cdacaterh
42,inferiortemporal,rh,-0.0132224478675366,0.0000,-2.28170639598336,H_site_smri_vol_cdk_iftmrh
57,precentral,rh,-0.00815004410075694,0.0000,-1.42088424100759,H_site_smri_vol_cdk_precnrh
59,rostralanteriorcingulate,rh,-0.0165446376906846,0.0000,-2.80994030154674,H_site_smri_vol_cdk_rracaterh


# Cognition - FA

In [101]:
Stats=matrix(nrow=length(c(grep('H_site_dmri_dtifa_fiberat',colnames(ABCD))[1:35])),ncol=6) # 
index=1

for (i in c(grep('H_site_dmri_dtifa_fiberat',colnames(ABCD))[1:35])) { #
  
    out <- summary(lmp(as.numeric(ABCD[[i]]) ~
                      ABCD$z.scores + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD, maxIter = 10000, center=F ))
    
        out2 <- summary(lm(as.numeric(ABCD[[i]]) ~
                      ABCD$z.scores + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F),
                      #factor(ABCD$site_id_l,ordered=F), 
                                data=ABCD))
    
    
   # name=unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[1]
   # hemi=unlist(strsplit(unlist(strsplit(unlist(strsplit(as.character(aliases$Aliases[which(aliases$ElementName==gsub('H_site_','',colnames(ABCD)[i]))]),'_'))[4],'[.]'))[2],','))[1]
    
    Stats[index,][1]=colnames(ABCD[i])
    Stats[index,][2]=''
    Stats[index,][3]=out$coefficients[2,1] # estimate
    Stats[index,][4]=out$coefficients[2,3] # p-value
    Stats[index,][5]=out2$coefficients[2,3] # t-value
    Stats[index,][6]=colnames(ABCD[i])
       
     
    index=index+1
}

Stats_adj=Stats
Stats_adj=data.frame(Stats_adj)
colnames(Stats_adj)=c('measure','hemi','estimate','p','t')
Stats_adj[,4]=as.numeric(p.adjust(Stats_adj[,4], method = 'BH'))

Stats_adj[Stats_adj[,4]<0.05,]

write.table(Stats,'Stats_corr_FAexec.csv', sep=",")

[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "
[1] "Settings:  unique SS "


,measure,hemi,estimate,p,t,NA
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
1,H_site_dmri_dtifa_fiberat_fxrh,,0.0391950090763142,0.0000000000,2.83854659815499,H_site_dmri_dtifa_fiberat_fxrh
2,H_site_dmri_dtifa_fiberat_fxlh,,0.0426993340414178,0.0000000000,2.90190309621449,H_site_dmri_dtifa_fiberat_fxlh
3,H_site_dmri_dtifa_fiberat_cgcrh,,0.0287751742311408,0.0000000000,1.9307893022903,H_site_dmri_dtifa_fiberat_cgcrh
4,H_site_dmri_dtifa_fiberat_cgclh,,0.0195636806689385,0.0129757785,1.36250943015415,H_site_dmri_dtifa_fiberat_cgclh
7,H_site_dmri_dtifa_fiberat_cstrh,,0.042435329896221,0.0000000000,3.0600604456802,H_site_dmri_dtifa_fiberat_cstrh
8,H_site_dmri_dtifa_fiberat_cstlh,,0.0507074515400123,0.0000000000,3.66946891834156,H_site_dmri_dtifa_fiberat_cstlh
9,H_site_dmri_dtifa_fiberat_atrrh,,0.0518724689505119,0.0000000000,3.93779455353031,H_site_dmri_dtifa_fiberat_atrrh
10,H_site_dmri_dtifa_fiberat_atrlh,,0.0522745969311346,0.0000000000,4.02945474826126,H_site_dmri_dtifa_fiberat_atrlh
11,H_site_dmri_dtifa_fiberat_uncrh,,0.0342376110998706,0.0000000000,2.55363246750463,H_site_dmri_dtifa_fiberat_uncrh


# Influence of ADI on PRS and BMI SDS

In [31]:
summary(lmp(ABCD$Z_corr_UKB ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                      reshist_addr1_coi_z_ed_nat +
                      reshist_addr1_coi_z_se_nat +
                      reshist_addr1_coi_z_he_nat, data=ABCD, maxIter = 10000, center=F))

summary(lmp(ABCD$BMI_SD ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 +
                     PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 +
                      scale(ABCD$interview_age) + 
                      factor(ABCD$sex, ordered=F) + 
                      factor(ABCD$demo_prnt_ed_v2, ordered=F) +
                      factor(ABCD$demo_comb_income_v2, ordered=F) + 
                      factor(ABCD$demo_ed_v2, ordered=F) +
                      factor(ABCD$demo_prnt_marital_v2, ordered=F) + 
                      reshist_addr1_coi_z_ed_nat +
                      reshist_addr1_coi_z_se_nat +
                      reshist_addr1_coi_z_he_nat, data=ABCD, maxIter = 10000, center=F))

[1] "Settings:  unique SS "



Call:
lmp(formula = ABCD$Z_corr_UKB ~ PC1 + PC2 + PC3 + PC4 + PC5 + 
    PC6 + PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + 
    PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + scale(ABCD$interview_age) + 
    factor(ABCD$sex, ordered = F) + factor(ABCD$demo_prnt_ed_v2, 
    ordered = F) + factor(ABCD$demo_comb_income_v2, ordered = F) + 
    factor(ABCD$demo_ed_v2, ordered = F) + factor(ABCD$demo_prnt_marital_v2, 
    ordered = F) + reshist_addr1_coi_z_ed_nat + reshist_addr1_coi_z_se_nat + 
    reshist_addr1_coi_z_he_nat, data = ABCD, center = F, maxIter = 10000)

Residuals:
      Min        1Q    Median        3Q       Max 
-4.031148 -0.649129  0.003751  0.671169  3.708681 

Coefficients:
                                                Estimate  Iter Pr(Prob)    
PC1                                              0.73450   866   0.1039    
PC2                                              1.20402    65   0.6154    
PC3                                              0.36917    51   1.000

[1] "Settings:  unique SS "



Call:
lmp(formula = ABCD$BMI_SD ~ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + 
    PC7 + Pc8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + 
    PC16 + PC17 + PC18 + PC19 + PC20 + scale(ABCD$interview_age) + 
    factor(ABCD$sex, ordered = F) + factor(ABCD$demo_prnt_ed_v2, 
    ordered = F) + factor(ABCD$demo_comb_income_v2, ordered = F) + 
    factor(ABCD$demo_ed_v2, ordered = F) + factor(ABCD$demo_prnt_marital_v2, 
    ordered = F) + reshist_addr1_coi_z_ed_nat + reshist_addr1_coi_z_se_nat + 
    reshist_addr1_coi_z_he_nat, data = ABCD, center = F, maxIter = 10000)

Residuals:
     Min       1Q   Median       3Q      Max 
-7.18632 -0.69081  0.01767  0.73823  2.83789 

Coefficients:
                                                Estimate  Iter Pr(Prob)    
PC1                                              4.39236 10000   <2e-16 ***
PC2                                             -2.34484    51   1.0000    
PC3                                              1.02288   834   0.1079    
PC4     